In [18]:
import pandas as pd
import os
import calendar
import re

# === SETUP AWAL ===
base_folder = r"C:\Users\Helga\semenindonesia.com\admpbp - PRODUKSI"
fmill_sheets = [f"FMILL#{i}" for i in range(1, 10)]
bulan_folder_valid = {"JAN", "PEB", "MAR", "APR", "MEI", "JUN", "JUL", "AGU", "SEP", "OKT", "NOV", "DES"}
valid_pattern_fmill = re.compile(r"^Tuban Fmill [a-z]{3}_25r\.xlsx$", re.IGNORECASE)

# === HEADER KOLABORASI AKHIR ===
fmill_full_header = [
    "Date", "Production (TPD)", "Production (TPH)", "Running Time",
    "Running Time (HRC)", "% Run HRC", "Number of Stops",
    "KWH MILL", "KWH SEPARATOR", "KWH Fan", "KWH MAIN", "KWH/TON",
    "TYPE SEMEN", "Cause of stop", "In Silo"
]

log_path = "log_merge_fmill.txt"
all_data_fmill = []

with open(log_path, "w") as log_file:
    log_file.write("=== LOG MERGE FINISH MILL ===\n\n")

# === LOOP PER BULAN ===
for subfolder in os.listdir(base_folder):
    if subfolder.upper() not in bulan_folder_valid:
        continue

    subfolder_path = os.path.join(base_folder, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    expected_file = f"tuban fmill {subfolder.lower()}_25r.xlsx"

    for file in os.listdir(subfolder_path):
        if not valid_pattern_fmill.match(file) or file.lower() != expected_file:
            continue

        full_path = os.path.join(subfolder_path, file)

        try:
            nama_bulan = subfolder.capitalize()
            bulan_num = list(calendar.month_abbr).index(nama_bulan[:3].capitalize())
            tahun = 2025
            jumlah_hari = calendar.monthrange(tahun, bulan_num)[1]
        except:
            continue

        try:
            xl = pd.ExcelFile(full_path)
        except Exception as e:
            with open(log_path, "a") as log_file:
                log_file.write(f"-- Gagal buka file: {file} - {e}\n")
            continue

        for sheet in fmill_sheets:
            if sheet not in xl.sheet_names:
                continue

            try:
                is_short = sheet in ["FMILL#1", "FMILL#2", "FMILL#3", "FMILL#4"]
                
                # FMILL#1–4: kolom excel G,H,I,J,K,L,M,N,U,V,W (indeks = 6,7,...)
                if is_short:
                    usecols = [6,7,8,9,10,11,12,13,20,21,22]  # 11 kolom
                    df = pd.read_excel(xl, sheet_name=sheet, skiprows=7, usecols=usecols, nrows=jumlah_hari, header=None)

                    df.columns = [
                        "Date", "Production (TPD)", "Production (TPH)", "KWH/TON",
                        "Running Time", "Running Time (HRC)", "% Run HRC", "Number of Stops",
                        "TYPE SEMEN", "Cause of stop", "In Silo"
                    ]

                    # Tambah kolom kosong yang tidak tersedia
                    df["KWH MILL"] = 0
                    df["KWH SEPARATOR"] = 0
                    df["KWH Fan"] = 0
                    df["KWH MAIN"] = 0

                    # Susun ulang agar urutan sesuai fmill_full_header
                    df = df[[
                        "Date", "Production (TPD)", "Production (TPH)", "Running Time",
                        "Running Time (HRC)", "% Run HRC", "Number of Stops",
                        "KWH MILL", "KWH SEPARATOR", "KWH Fan", "KWH MAIN", "KWH/TON",
                        "TYPE SEMEN", "Cause of stop", "In Silo"
                    ]]
                
                else:
                    # FMILL#5–9: kolom G,H,I,K,L,M,N,O,P,Q,S,T,U,V,W → indeks [6,7,8,10,...]
                    usecols = [6,7,8,10,11,12,13,14,15,16,18,19,20,21,22]
                    df = pd.read_excel(xl, sheet_name=sheet, skiprows=7, usecols=usecols, nrows=jumlah_hari, header=None)

                    df.columns = fmill_full_header

                # Hapus baris kosong
                df.dropna(how="all", inplace=True)

                # Validasi dan parsing tanggal
                df["Date"] = pd.to_numeric(df["Date"], errors="coerce")
                df = df[df["Date"].notnull() & df["Date"].between(1, jumlah_hari)]

                df["tanggal"] = pd.to_datetime({
                    "year": tahun,
                    "month": bulan_num,
                    "day": df["Date"].astype(int)
                })

                df["FMILL"] = sheet
                df["Bulan"] = nama_bulan
                df["SourceFilePath"] = full_path

                all_data_fmill.append(df)
                print(f"-- Dibaca: {file} - {sheet}")
                with open(log_path, "a") as log_file:
                    log_file.write(f"-- Dibaca: {file} - {sheet}\n")

            except Exception as e:
                print(f"-- Gagal membaca {file} - {sheet}: {e}")
                with open(log_path, "a") as log_file:
                    log_file.write(f"-- Gagal membaca {file} - {sheet}: {e}\n")

# === SIMPAN ===
if all_data_fmill:
    hasil = pd.concat(all_data_fmill, ignore_index=True)
    hasil.sort_values(by=["tanggal", "FMILL"], inplace=True)

    output_path = r"D:\MAGANG SIG\TUGAS 2 - MERGE FMILL\FMILL_MERGE_FINAL.xlsx"
    hasil.to_excel(output_path, index=False)

    print(f"\n-- Berhasil disimpan di: {output_path}")
    with open(log_path, "a") as log_file:
        log_file.write(f"\n-- Berhasil disimpan di: {output_path}\n")
else:
    print("\n-- Tidak ada data berhasil dibaca.")
    with open(log_path, "a") as log_file:
        log_file.write("\n-- GAGAL: Tidak ada data berhasil dibaca.\n")


-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#1
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#2
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#3
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#4
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#5
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#6
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#7
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#8
-- Dibaca: Tuban Fmill apr_25r.xlsx - FMILL#9
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#1
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#2
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#3
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#4
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#5
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#6
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#7
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#8
-- Dibaca: Tuban Fmill jan_25r.xlsx - FMILL#9
-- Dibaca: Tuban Fmill jul_25r.xlsx - FMILL#1
-- Dibaca: Tuban Fmill jul_25r.xlsx - FMILL#2
-- Dibaca: Tuban Fmill jul_25r.xlsx - FMILL#3
-- Dibaca: Tuban Fmill jul_25r.xls

* Menambahkan Data Terbaru 

In [ ]:
import pandas as pd
import os
import calendar
import re

# === SETUP ===
base_folder = r"C:\Users\Helga\semenindonesia.com\admpbp - PRODUKSI"
output_path = r"D:\MAGANG SIG\TUGAS 2 - MERGE FMILL\FMILL_MERGE_FINAL.xlsx"
log_path = "log_merge_fmill.txt"
processed_file_record = "processed_files.txt"

fmill_sheets = [f"FMILL#{i}" for i in range(1, 10)]
bulan_folder_valid = {"JAN", "PEB", "MAR", "APR", "MEI", "JUN", "JUL", "AGU", "SEP", "OKT", "NOV", "DES"}
valid_pattern_fmill = re.compile(r"^Tuban Fmill [a-z]{3}_25r\.xlsx$", re.IGNORECASE)

fmill_full_header = [
    "Date", "Production (TPD)", "Production (TPH)", "Running Time",
    "Running Time (HRC)", "% Run HRC", "Number of Stops",
    "KWH MILL", "KWH SEPARATOR", "KWH Fan", "KWH MAIN", "KWH/TON",
    "TYPE SEMEN", "Cause of stop", "In Silo"
]

# === BACA FILE YANG SUDAH PERNAH DIPROSES ===
if os.path.exists(processed_file_record):
    with open(processed_file_record, "r") as f:
        processed_files = set(line.strip() for line in f)
else:
    processed_files = set()

new_data = []
with open(log_path, "w") as log_file:
    log_file.write("=== LOG PROSES DATA BARU FINISH MILL ===\n\n")

# === LOOP FOLDER BULAN ===
for subfolder in os.listdir(base_folder):
    if subfolder.upper() not in bulan_folder_valid:
        continue

    subfolder_path = os.path.join(base_folder, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    expected_file = f"tuban fmill {subfolder.lower()}_25r.xlsx"

    for file in os.listdir(subfolder_path):
        if not valid_pattern_fmill.match(file) or file.lower() != expected_file:
            continue

        full_path = os.path.join(subfolder_path, file)
        file_id = f"{subfolder}/{file}"

        if file_id in processed_files:
            continue  # sudah diproses sebelumnya

        try:
            nama_bulan = subfolder.capitalize()
            bulan_num = list(calendar.month_abbr).index(nama_bulan[:3].capitalize())
            tahun = 2025
            jumlah_hari = calendar.monthrange(tahun, bulan_num)[1]
        except:
            continue

        try:
            xl = pd.ExcelFile(full_path)
        except Exception as e:
            with open(log_path, "a") as log_file:
                log_file.write(f"-- Gagal buka file: {file} - {e}\n")
            continue

        for sheet in fmill_sheets:
            if sheet not in xl.sheet_names:
                continue

            try:
                is_short = sheet in ["FMILL#1", "FMILL#2", "FMILL#3", "FMILL#4"]

                if is_short:
                    usecols = [6,7,8,9,10,11,12,13,20,21,22]  # G,H,I,J,K,L,M,N,U,V,W
                    df = pd.read_excel(xl, sheet_name=sheet, skiprows=7, usecols=usecols, nrows=jumlah_hari, header=None)
                    df.columns = [
                        "Date", "Production (TPD)", "Production (TPH)", "KWH/TON",
                        "Running Time", "Running Time (HRC)", "% Run HRC", "Number of Stops",
                        "TYPE SEMEN", "Cause of stop", "In Silo"
                    ]
                    df["KWH MILL"] = 0
                    df["KWH SEPARATOR"] = 0
                    df["KWH Fan"] = 0
                    df["KWH MAIN"] = 0
                    df = df[[
                        "Date", "Production (TPD)", "Production (TPH)", "Running Time",
                        "Running Time (HRC)", "% Run HRC", "Number of Stops",
                        "KWH MILL", "KWH SEPARATOR", "KWH Fan", "KWH MAIN", "KWH/TON",
                        "TYPE SEMEN", "Cause of stop", "In Silo"
                    ]]
                else:
                    usecols = [6,7,8,10,11,12,13,14,15,16,18,19,20,21,22]  # G,H,I,K,L,M,N,O,P,Q,S,T,U,V,W
                    df = pd.read_excel(xl, sheet_name=sheet, skiprows=7, usecols=usecols, nrows=jumlah_hari, header=None)
                    df.columns = fmill_full_header

                df.dropna(how="all", inplace=True)
                df["Date"] = pd.to_numeric(df["Date"], errors="coerce")
                df = df[df["Date"].notnull() & df["Date"].between(1, jumlah_hari)]

                df["tanggal"] = pd.to_datetime({
                    "year": tahun,
                    "month": bulan_num,
                    "day": df["Date"].astype(int)
                })

                df["FMILL"] = sheet
                df["Bulan"] = nama_bulan
                df["SourceFilePath"] = full_path

                new_data.append(df)

                print(f"-- Dibaca: {file} - {sheet}")
                with open(log_path, "a") as log_file:
                    log_file.write(f"-- Dibaca: {file} - {sheet}\n")

            except Exception as e:
                print(f"-- Gagal membaca {file} - {sheet}: {e}")
                with open(log_path, "a") as log_file:
                    log_file.write(f"-- Gagal membaca {file} - {sheet}: {e}\n")

        # Tambahkan ke daftar processed
        processed_files.add(file_id)

# === SIMPAN HASIL ===
if new_data:
    df_new = pd.concat(new_data, ignore_index=True)
    df_new.sort_values(by=["tanggal", "FMILL"], inplace=True)

    if os.path.exists(output_path):
        df_existing = pd.read_excel(output_path)
        df_result = pd.concat([df_existing, df_new], ignore_index=True)
        df_result.drop_duplicates(subset=["tanggal", "FMILL"], keep="last", inplace=True)
    else:
        df_result = df_new

    df_result.sort_values(by=["tanggal", "FMILL"], inplace=True)
    df_result.to_excel(output_path, index=False)

    # Simpan daftar file yang sudah diproses
    with open(processed_file_record, "w") as f:
        for item in sorted(processed_files):
            f.write(item + "\n")

    print(f"\n-- Berhasil disimpan di: {output_path}")
    with open(log_path, "a") as log_file:
        log_file.write(f"\n-- Berhasil disimpan di: {output_path}\n")
else:
    print("\n-- Tidak ada file baru yang perlu diproses.")
    with open(log_path, "a") as log_file:
        log_file.write("\n-- Tidak ada file baru yang perlu diproses.\n")
